In [ ]:
print("="*80)
print("SETTING UP ENVIRONMENT")
print("="*80)

# Install required packages
import sys
import subprocess

def install_packages():
    """Install required packages for calibration analysis."""
    packages = [
        'scipy>=1.10.0',
        'scikit-learn>=1.3.0',
        'matplotlib>=3.7.0',
        'seaborn>=0.12.0',
        'numpy>=1.24.0',
    ]

    for package in packages:
        print(f"Installing {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", package])

    print("All packages installed!")

# Uncomment to install (run once)
# install_packages()


In [ ]:
# ============================================================================
# CELL 2: Imports
# ============================================================================
import numpy as np
import json
from pathlib import Path
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.calibration import calibration_curve
from sklearn.metrics import confusion_matrix, roc_curve, auc, roc_auc_score
import matplotlib.pyplot as plt
import seaborn as sns

print(" All imports loaded")


In [ ]:
# ============================================================================
# CELL 3: Configuration
# ============================================================================
print("\n" + "="*80)
print("CONFIGURATION")
print("="*80)

class Config:
    """Calibration analysis configuration (Hypothesis 3)."""
    # Path to style probing results (from H1 notebook)
    style_probing_dir = "./results/style_probing"
    
    # Random seed for reproducibility
    seed = 42
    
    # Output directory
    output_dir = "./results/style_probing"

config = Config()

print(f"Style probing results directory: {config.style_probing_dir}")
print(f"Random seed: {config.seed}")
print(f"Output directory: {config.output_dir}")


In [ ]:
# ============================================================================
# CELL 4: Load Features from H1 Results
# ============================================================================
print("\n" + "="*80)
print("LOADING FEATURES")
print("="*80)

output_dir = Path(config.style_probing_dir)
features_file = output_dir / "raw_features.npz"

if not features_file.exists():
    raise FileNotFoundError(
        f"Features file not found: {features_file}\n"
        "Please run H1 notebook first to generate features."
    )

# Load features
data = np.load(features_file, allow_pickle=True)
X = data['X']
y = data['y']
features_us = data['features_us']
features_uk = data['features_uk']
feature_names = data['feature_names'].tolist()

print(f" Loaded feature matrix: {X.shape}")
print(f" Number of features: {len(feature_names)}")
print(f" US samples: {len(features_us)}")
print(f" UK samples: {len(features_uk)}")


In [ ]:
# ============================================================================
# CELL 5: Cohort Recoverability (5-fold CV)
# ============================================================================
print("\n" + "="*80)
print("COHORT RECOVERABILITY ANALYSIS")
print("="*80)

# Set random seed
np.random.seed(config.seed)

# Train logistic regression classifier with cross-validation
classifier = LogisticRegression(max_iter=1000, random_state=config.seed)

# 5-fold cross-validation
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=config.seed)
cv_scores = cross_val_score(classifier, X, y, cv=cv, scoring='accuracy')

print(f"5-fold CV Accuracy: {cv_scores.mean():.4f} (+/- {cv_scores.std() * 2:.4f})")
print(f"Individual fold scores: {cv_scores}")

# Train on full dataset for analysis
classifier.fit(X, y)
predictions = classifier.predict(X)
probabilities = classifier.predict_proba(X)[:, 1]  # Probability of class 1 (UK)

print(f"\n Classifier trained on full dataset")
print(f"  Training accuracy: {classifier.score(X, y):.4f}")
print(f"  Chance level: 0.5")


In [ ]:
# ============================================================================
# CELL 6: Calibration Plot
# ============================================================================
print("\n" + "="*80)
print("GENERATING CALIBRATION PLOT")
print("="*80)

def plot_calibration_curve(y_true, y_prob, output_path):
    """Plot calibration curve for cohort classifier."""
    fraction_of_positives, mean_predicted_value = calibration_curve(
        y_true, y_prob, n_bins=10, strategy='uniform'
    )
    
    plt.figure(figsize=(8, 6))
    plt.plot(mean_predicted_value, fraction_of_positives, "s-", label="Cohort Classifier")
    plt.plot([0, 1], [0, 1], "k--", label="Perfectly Calibrated")
    plt.xlabel("Mean Predicted Probability")
    plt.ylabel("Fraction of Positives")
    plt.title("Calibration Plot: Cohort Recoverability (H3)")
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig(output_path, dpi=300, bbox_inches='tight')
    plt.close()
    print(f" Calibration plot saved to {output_path}")

calibration_path = output_dir / "h3_calibration_plot.png"
plot_calibration_curve(y, probabilities, calibration_path)


In [ ]:
# ============================================================================
# CELL 7: Classifier Analysis (Confusion Matrix, ROC, Feature Importance)
# ============================================================================
print("\n" + "="*80)
print("GENERATING CLASSIFIER ANALYSIS")
print("="*80)

def plot_classifier_analysis(classifier, X, y, feature_names, output_dir):
    """Plot classifier analysis for H3: confusion matrix, ROC, feature importance."""
    
    # Get predictions and probabilities
    y_pred = classifier.predict(X)
    y_prob = classifier.predict_proba(X)[:, 1]
    
    # Create figure with subplots
    fig = plt.figure(figsize=(16, 5))
    
    # 1. Confusion Matrix
    ax1 = plt.subplot(1, 3, 1)
    cm = confusion_matrix(y, y_pred)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax1,
                xticklabels=['US', 'UK'], yticklabels=['US', 'UK'])
    ax1.set_xlabel('Predicted')
    ax1.set_ylabel('Actual')
    ax1.set_title('Confusion Matrix (H3)')
    
    # 2. ROC Curve
    ax2 = plt.subplot(1, 3, 2)
    fpr, tpr, _ = roc_curve(y, y_prob)
    roc_auc = roc_auc_score(y, y_prob)
    ax2.plot(fpr, tpr, color='darkorange', lw=2, 
             label=f'ROC curve (AUC = {roc_auc:.3f})')
    ax2.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random')
    ax2.set_xlabel('False Positive Rate')
    ax2.set_ylabel('True Positive Rate')
    ax2.set_title('ROC Curve (H3)')
    ax2.legend(loc="lower right")
    ax2.grid(True, alpha=0.3)
    
    # 3. Feature Importance (coefficient magnitudes)
    ax3 = plt.subplot(1, 3, 3)
    coef = classifier.coef_[0]
    feature_importance = np.abs(coef)
    top_indices = np.argsort(feature_importance)[-10:][::-1]
    
    top_features = [feature_names[i] for i in top_indices]
    top_importance = feature_importance[top_indices]
    
    ax3.barh(range(len(top_features)), top_importance, color='steelblue', alpha=0.7)
    ax3.set_yticks(range(len(top_features)))
    ax3.set_yticklabels([f.replace('_', ' ').title() for f in top_features])
    ax3.set_xlabel('|Coefficient| (Feature Importance)')
    ax3.set_title('Top 10 Most Important Features (H3)')
    ax3.grid(True, alpha=0.3, axis='x')
    
    plt.tight_layout()
    output_path = output_dir / "classifier_analysis.png"
    plt.savefig(output_path, dpi=300, bbox_inches='tight')
    plt.close()
    print(f" Classifier analysis plot saved to {output_path}")

plot_classifier_analysis(classifier, X, y, feature_names, output_dir)


In [ ]:
# ============================================================================
# CELL 8: Save Results
# ============================================================================
print("\n" + "="*80)
print("SAVING RESULTS")
print("="*80)

# Compute ROC AUC
fpr, tpr, _ = roc_curve(y, probabilities)
roc_auc = roc_auc_score(y, probabilities)

# Compute confusion matrix
cm = confusion_matrix(y, predictions)

results = {
    'hypothesis': 'H3',
    'cohort_recoverability': {
        'cv_accuracy_mean': float(cv_scores.mean()),
        'cv_accuracy_std': float(cv_scores.std()),
        'cv_scores': cv_scores.tolist(),
        'training_accuracy': float(classifier.score(X, y)),
        'chance_level': 0.5,
    },
    'roc_auc': float(roc_auc),
    'confusion_matrix': cm.tolist(),
    'top_features': {
        feature_names[i]: float(np.abs(classifier.coef_[0][i]))
        for i in np.argsort(np.abs(classifier.coef_[0]))[-10:][::-1]
    }
}

results_path = output_dir / "h3_results.json"
with open(results_path, 'w') as f:
    json.dump(results, f, indent=2)

print(f" Results saved to {results_path}")
print(f"\nSummary:")
print(f"  CV Accuracy: {cv_scores.mean():.4f} (+/- {cv_scores.std() * 2:.4f})")
print(f"  ROC AUC: {roc_auc:.4f}")
print(f"  Training Accuracy: {classifier.score(X, y):.4f}")


In [ ]:
# ============================================================================
# CELL 9: Summary
# ============================================================================
print("\n" + "="*80)
print("HYPOTHESIS 3 EVALUATION COMPLETE")
print("="*80)
print(f"\nResults saved to: {output_dir}")
print(f"  - Calibration plot: {calibration_path}")
print(f"  - Classifier analysis: {output_dir / 'classifier_analysis.png'}")
print(f"  - Results: {results_path}")
print(f"\nCohort classifier CV accuracy: {cv_scores.mean():.4f} (chance: 0.5)")
print(f"ROC AUC: {roc_auc:.4f}")

if cv_scores.mean() > 0.5:
    print("\n Hypothesis 3 supported: Models show stylistic divergence detectable by classifier!")
else:
    print("\n Hypothesis 3 not supported: No clear stylistic divergence detected.")
